# Word info synthesis

In [43]:
from ipynb.fs.full.koselleck import *

In [44]:
def get_word_comparisons(word):
    with get_veclib('ttest') as vl_ttest_vec, get_veclib('ttest_lnm') as vl_ttest_lnm:
        wdf_ttest_vec=vl_ttest_vec.get(word,pd.DataFrame())
        wdf_ttest_lnm=vl_ttest_lnm.get(word,pd.DataFrame())
        odf=wdf_ttest_vec.append(wdf_ttest_lnm)
        if 'index' in set(odf.columns):odf=odf.drop('index',1)
        if len(odf):
            odf['avg_div']=odf.avg2/odf.avg1
            odf['avg_diff_perc']=odf.avg2_perc_vec - odf.avg1_perc_vec
            odf['avg_diff_perc_abs']=odf['avg_diff_perc'].apply(abs)
        return odf

In [46]:
get_word_comparisons('sympathy').loc['sympathy','LNM']

,n1,n2,avg1,avg2,avg_diff,mw,mw_p,mw_perc,mw_perc_vec,mw_perc_vec_cmp,avg1_perc_vec,avg2_perc_vec,avg_div,avg_diff_perc,avg_diff_perc_abs
period_cmp,,,,,,,,,,,,,,,
After-v-After,314.0,91.0,1.896794,-1.096069,-2.992864,1287.0,3.312834e-40,93.486328,93.486328,89.816975,7.643229,84.537760,-0.577853,76.894531,76.894531
During-v-During,339.0,66.0,1.425331,0.191879,-1.233452,7905.0,8.109481e-05,46.399740,46.399740,58.435984,13.470052,19.348958,0.134620,5.878906,5.878906


In [56]:
def info_word_comparisons(word,veckey='vector'):
    df=get_word_comparisons(word).reset_index().dropna()
    df.loc[df.vector=='LNM', 'avg_diff_perc'] *= -1
    
    most_changing_vecs = df.groupby(veckey).mean().reset_index().sort_values('avg_diff_perc_abs',ascending=False).drop_duplicates(veckey)[veckey]
#     print(most_changing_vecs)
    o=[]
    for vec in most_changing_vecs:
        vecdf=df.query(f'vector=="{vec}"').set_index('period_cmp')
        
        if vec!='LNM':
            bef_dur=vecdf.loc['Before-v-During']
            dur_aft=vecdf.loc['During-v-After']
            bef_aft=vecdf.loc['Before-v-After']
            
        else:
            bef_dur=vecdf.loc['Before-v-Before']
            dur_aft=vecdf.loc['During-v-During']
            bef_aft=vecdf.loc['After-v-After']

        

    #         perc1,perc2,perc3 = bef_dur.avg1_perc_vec, bef_dur.avg2_perc_vec, dur_aft.avg2_perc_vec
        perc1,perc2,perc3 = bef_dur.avg1, bef_dur.avg2, dur_aft.avg2
        change=bef_aft.avg_diff
        is_signif=bef_aft.mw_p<.01
        mwperc=bef_aft.mw_perc
        mwp=bef_aft.mw_p
        
        
        vname=vec.split(".")[0]
        
        if '-' in vname:
            vname1,vname2=vname.split('-',1)
        else:
            vname1,vname2=vname,vname
        
        change2=abs(change) if vname2!=vname1 else change
        vname2_ch=vname2 if vname2!=vname1 else vname1
#         if vname=='LNM': change*=-1
        
        is_pos_change=change>0
        
        vname_change=vname1 if is_pos_change else vname2_ch
#         changestr=f'{change:+.0f}% {vname1}' if is_pos_change else f'{change2:+.0f}% {vname2_ch}'
        changestr=f'{abs(change):.2f}' if is_pos_change else f'{abs(change):.2f}'

        
#         x=f'* <u>{vname}</u>: {changestr} ({perc1:.0f}% -> {perc2:.0f}% -> {perc3:.0f}%)'
        vnameA = vname1 if perc1>0 else vname2
        vnameB = vname1 if perc2>0 else vname2
        vnameC = vname1 if perc3>0 else vname2
#         ltr1=f"{'+' if is_pos_change else '-'}"
        ltr1='+'
        vchange=f"""{ltr1 if vnameA!=vnameC else ''}{ltr1}{vname_change}"""
        if is_signif: 
            vchange=f'<b>{vchange}</b>'
            changestr=f'{changestr}'
        if vnameB!=vnameA: vnameB=f'<b>{vnameB}</b>'
        if vnameC!=vnameB: vnameC=f'<b>{vnameC}</b>'
            
#         perc1b=(100-perc1 if perc1>=50 else perc1)
#         perc2b=(100-perc2 if perc2>=50 else perc2)
#         perc3b=(100-perc3 if perc3>=50 else perc3)
#         perc1b=abs(perc1-50)*2
#         perc2b=abs(perc2-50)*2
#         perc3b=abs(perc3-50)*2
        
        
        
        
#         x=f'_{vname}_ | <u>{vchange}</u> ({changestr}) | {perc1:.0f} {perc1b:.0f}% {vnameA} | {perc2b:.0f}% {vnameB} | {perc3:.0f} {perc3b:.0f}% {vnameC}'
        x=f'_{vname}_ | <u>{vchange}</u> ({changestr}) | {perc1:+.2f} ({vnameA}) | {perc2:+.2f} ({vnameB}) | {perc3:+.2f} ({vnameC})'
        o+=[x]
    
    o=sorted(o,key=lambda x: float(x.split('|')[1].strip().split()[-1].replace(')','').replace('(','')),reverse=True)
    
    hdr="""
| Vector | Change | 1700-1770 | 1770-1830 | 1830-1900 |
| ------ | --------- | --------- | --------- | ------ |
    """
    ostr = hdr + '\n'.join(o)
    return ostr

printm(info_word_comparisons('culture'))    


| Vector | Change | 1700-1770 | 1770-1830 | 1830-1900 |
| ------ | --------- | --------- | --------- | ------ |
    _Virtue-Vice_ | <u><b>+Virtue</b></u> (1.89) | +1.10 (Virtue) | +1.25 (Virtue) | +3.00 (Virtue)
_Pos-Neg_ | <u><b>+Pos</b></u> (1.69) | +1.05 (Pos) | +1.18 (Pos) | +2.74 (Pos)
_Human-Object_ | <u><b>++Human</b></u> (1.62) | -1.42 (Object) | -1.44 (Object) | +0.20 (<b>Human</b>)
_Abs-Conc_ | <u><b>+Abs</b></u> (1.58) | +0.10 (Abs) | +0.27 (Abs) | +1.68 (Abs)
_Pleasure-Pain_ | <u><b>+Pleasure</b></u> (1.53) | +0.88 (Pleasure) | +1.06 (Pleasure) | +2.42 (Pleasure)
_Interp-Desc_ | <u><b>+Interp</b></u> (1.48) | +0.65 (Interp) | +1.04 (Interp) | +2.13 (Interp)
_Strong-Weak_ | <u><b>+Strong</b></u> (1.20) | +0.81 (Strong) | +1.38 (Strong) | +2.00 (Strong)
_Time-Space_ | <u><b>+Time</b></u> (0.84) | -1.12 (Space) | -0.75 (Space) | -0.27 (Space)
_Polit-Acad_ | <u><b>+Acad</b></u> (0.81) | -1.01 (Acad) | -1.10 (Acad) | -1.82 (Acad)
_Qual-Quant_ | <u><b>+Quant</b></u> (0.64) | -0.38 (Quant) | -1.27 (Quant) | -1.02 (Quant)
_LNM_ | <u><b>+LNM</b></u> (0.55) | +0.32 (LNM) | +1.36 (LNM) | +0.50 (LNM)
_Woman-Man_ | <u><b>+Man</b></u> (0.47) | -0.47 (Man) | -0.98 (Man) | -0.94 (Man)
_Freq_ | <u><b>+Freq</b></u> (0.40) | -0.23 (Freq) | -0.03 (Freq) | +0.16 (Freq)
_Active-Passive_ | <u><b>++Passive</b></u> (0.31) | +0.08 (Active) | +0.65 (Active) | -0.23 (<b>Passive</b>)
_Ambig_ | <u><b>+Ambig</b></u> (0.20) | -0.02 (Ambig) | -0.30 (Ambig) | +0.18 (Ambig)
_Collective-Indiv_ | <u>+Collective</u> (0.08) | +1.54 (Collective) | +1.65 (Collective) | +1.62 (Collective)
_Judg-Perc_ | <u>+Perc</u> (0.03) | -0.31 (Perc) | -0.42 (Perc) | -0.34 (Perc)